## Sentiment 사용법 (감성분석 및 키워드 추출)

## 문장 추출

In [ ]:
import pandas as pd
import Sentiment
import sqlite3

- 감성분석할 Data load

In [ ]:
# new_words_var.xlsx에서 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

- 해당 용어가 추출된 전체 Dataset load

In [ ]:
# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

- 해당 용어가 들어있는 Sentence 추출

In [ ]:
analysis = Sentiment.Sentiment()
sent = analysis.extract_sent(head, new_words) # DataFrame(head)과 신어(words) 입력하면 신어에 해당하는 sentences 추출
sent = pd.DataFrame.from_dict(sent, orient='index') # Pandas DataFrame으로 저장
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn) # 신어와 신어가 들어있는 예문을 sent.db로 저장

## 감성분석

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

- instance화 및 Data load

In [ ]:
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)
mapping_most_related = analysis.cal_score(sent)
# 신어-문장으로 된 DataFrame(sent) 입력하면 sentiment 점수를 excel에 저장하고 신조어와 most_related (단어, pmi) 쌍을 출력
# 다음 감성사전 업데이트를 효율적으로 하기 위해서 그 결과를 mapping_most_related에 저장한다.

- 감성사전 업데이트 및 감성점수 추출

In [ ]:
positive = []
negative = []
for _ in tqdm(range(40)):
    # 일정 (단어-감성score) set을 사용하여, 감성사전 score 업데이트 (학습과정)
    sentiment_result = pd.read_excel('sentiment_result.xlsx') # sentiment_result.xlsx : 감성점수 엑셀 파일
    positive, negative = analysis.update_score(positive, negative, sentiment_result) 
    # 감성점수를 이용하여, 감성사전 score를 업데이트한다.(sentiment.xlsx가 업데이트 됨)
    
    # 감성사전을 이용하여 감성분석을 진행
    sentiment = pd.read_excel('sentiment.xlsx')
    analysis.sentiment_analysis(sentiment, mapping_most_related)
    # 업데이트된 감성 점수를 이용하여 감성분석을 다시 진행한다.

- 추출한 감성점수를 정규분포를 이용하여 percentage로 나타내기

In [ ]:
score = pd.read_excel('sentiment_result.xlsx') # 감성분석 결과 가져오기

# 정규화
temp = np.array(score[1])
temp = StandardScaler().fit_transform(temp.reshape(-1,1))

# 정규분포 누적확률 구하기
temp = norm.cdf(temp)*100

# 긍정, 부정으로 DataFrame에 집어넣기
score[1] = temp
score[2] = 100-score[1]
score.rename({0:'Word', 1:'Positive', 2:'Negative'}, axis=1, inplace=True)

score.to_excel('News_sentiment_result.xlsx') # 저장

## 키워드 추출

In [ ]:
import Sentiment
import sqlite3
import pandas as pd
from collections import defaultdict
import ast

- Data load

In [ ]:
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)

- most_similar 함수를 이용하여 유사하고 (ktemp), 관련있는 (rtemp) 데이터 가져오기

In [ ]:
ktemp = defaultdict(lambda:0)
rtemp = defaultdict(lambda:0)
# 모든 신어에 대해서 진행
for _ in range(len(sent)):
    words = {sent.iloc[_]['index']:1.0} # 각 신어를 하나의 단어로 인식할 수 있도록 dict 생성
    keywords = analysis.most_similar(sent.iloc[_]['index'], sent.iloc[_]['0'], words)
    ktemp[sent.iloc[_]['index']] = keywords[0] # 유사한 용어
    rtemp[sent.iloc[_]['index']] = keywords[1] # 관련있는 용어 (같이 많이 쓰이는 용어)
pd.DataFrame.from_dict(ktemp, orient='index').to_excel('keyword.xlsx') # excel로 저장
pd.DataFrame.from_dict(rtemp, orient='index').to_excel('rkeyword.xlsx') # excel로 저장

- 위에서 생성한 Data를 이용하여, 가장 관련있는 Keyword 5개 뽑기

In [ ]:
# Data load
keyword = pd.read_excel('keyword.xlsx')
rkeyword = pd.read_excel('rkeyword.xlsx')
df = pd.merge(keyword, rkeyword, on='Unnamed: 0')

key = defaultdict(lambda:0)
# 신어 : 연관키워드 쌍의 Dictionary 형태로 변환
for i in range(len(df)):
    temp = [j for j in df.iloc[i,:] if type(j) == str]
    key[temp[0]] = temp[1:]
    
# 그 중 점수가 가장 높은 5개 추출
for k,v in key.items():
    temp = [ast.literal_eval(_) for _ in v]
    temp = sorted(temp, key=lambda _:_[-1], reverse=True)
    while len(temp) > 5:
        temp.pop(-1)
    key[k] = temp

pd.DataFrame.from_dict(key, orient='index').to_excel('key.xlsx') # key.xlsx에 결과 저장